# import

In [1]:

import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
# import importlib
# importlib.reload(hoge)
from my_library.horse import *
import pickle


# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_win2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_win = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'
path_ft = '/Users/Owner/Desktop/Horse/ft_data/peds_ft.txt'

# load data

In [3]:
results = load_csv(path_win+'results.csv')
horse_results = load_csv(path_win+'horse_results.csv')
peds = load_csv(path_win+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_win+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [150]:
ll.hr

In [152]:
ll = LearnLGBM(peds,results,horse_results)
ll.path_ft = '/Users/Owner/Desktop/Horse/horse/peds_ft.txt'
ll.learn_lgb()

  0%|          | 0/18673 [00:00<?, ?it/s]

pe finish
pe regularizrd


  0%|          | 0/277 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 76081
[LightGBM] [Info] Number of data points in the train set: 96940, number of used features: 172


# Simulate

In [153]:
class Simulater():
    
    def __init__(self, model):
        self.model = model
        self.return_tables = None
        self.pred_df = None
        self.is_long = True
    

    #     当日のデータでシミュレートするとあかん
    def return_table(self, race_id_list):
        return_tables = Return.scrape(race_id_list)
        return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
        self.return_tables = return_tables
    
    
    def return_table_today(self,race_id_list):
        return_tables = {}
        for race_id in tqdm(race_id_list):
            try:
                url = 'https://race.netkeiba.com/race/result.html?race_id='+race_id+'&amp;rf=race_submenu'
                dfs = pd.read_html(url)
                df = pd.concat([dfs[1], dfs[2]])
                df.index = [race_id] * len(df)
                return_tables[race_id] = df
                time.sleep(0.5)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
            #pd.DataFrame型にして一つのデータにまとめる
        return_tables_df = pd.concat([return_tables[key] for key in return_tables])
        # return_tables_df.index = return_tables_df.index.astype(int)
        self.return_tables = return_tables_df
    
    
    def return_pred_table(self,data_c,is_long=False):
        # is_long って何？
        #予測
        if not is_long:
            scores = pd.Series(self.model.predict(data_c.drop(['date'],axis=1)),index=data_c.index)
        else:
            scores = pd.Series(self.model.predict(data_c.drop(['date','rank','単勝'],axis=1)),index=data_c.index)
        pred = data_c[['馬番']].copy()
        pred['scores'] = scores
        pred = pred.sort_values('scores',ascending=False)
        return pred

#     odds以上の馬券しか買わない
    def get_result_df(self, data_c, return_tables, is_long=True, odds=2.0, bet = 100):
        race_id_list = list(set(data_c.index))
        race_dict = {}

        for race_id in race_id_list:
            pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb,odds_list = self.return_race_result(data_c,race_id,return_tables)
            row_list = [pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb,odds_list]
            race_dict[int(race_id)] = row_list
        all_result = pd.DataFrame(race_dict).T
        all_result.rename(columns={
            0:'pred_list',
            1:'actual_rank_list',
            2:'tansho_odds',
            3:'fukusho_odds',
            4:'umaren_odds',
            5:'wide_odds',
            6:'umatan_odds',
            7:'sanrenpuku_odds',
            8:'sanrentan_odds',
            9:'wide_comb',
            10:'odds_list'
            },inplace=True)
        return all_result
        

    def return_race_result(self, data_c ,race_id, return_tables):
        race_id = int(race_id)
        pred_df = self.return_pred_table(data_c.loc[race_id],is_long=self.is_long)
        pred_df = pred_df.loc[race_id]
        pred_df = pred_df.sort_values('scores',ascending=False)
        dc = data_c.loc[race_id]
        return_table  = return_tables.loc[race_id]
        
        
        pred_list = [int(pred_df['馬番'].iloc[i]) for i in range(len(pred_df))]

        score_1 = pred_df['scores'].iloc[0]
        score_2 = pred_df['scores'].iloc[1]
        is_same_score = False

        try:
        
            tansho_row = return_table[return_table[0]=='単勝']
            fukusho_row = return_table[return_table[0]=='複勝']
            umaren_row =  return_table[return_table[0]=='馬連']
            umatan_row =  return_table[return_table[0]=='馬単']
            wide_row =  return_table[return_table[0]=='ワイド']
            sanrentan_row =  return_table[return_table[0]=='三連単']
            sanrenpuku_row =  return_table[return_table[0]=='三連複']
            
            # odds 順番は予測した順
            odds_list = []
            for ub in pred_df['馬番'].tolist():
                odds_list.append(dc[dc['馬番']==ub]['単勝'].values[0])
        
            if score_1 == score_2:
                is_same_score =True
                
            # １着が同着    
            if int(tansho_row[1].str.count('br'))==1:
                actual_tmp0 = sanrentan_row[1].str.split('br').values[0][0]
                actual_tmp1 = sanrentan_row[1].str.split('br').values[0][1]
                actual_rank_list0 = list(map(int,actual_tmp0.split('→')))
                actual_rank_list1 = list(map(int,actual_tmp1.split('→')))
                actual_rank_list = [actual_rank_list0,actual_rank_list1]
                
                tansho_odds_list = tansho_row[2].str.split('br').values[0][0:3]
                tansho_odds_list = [i for i in tansho_odds_list if i!='']
                tansho_odds = list(map(lambda x: int(x.replace(',',''))/100 ,tansho_odds_list))
                
                umatan_odds_list = umatan_row[2].str.split('br').values[0][0:3]
                umatan_odds_list = [i for i in umatan_odds_list if i!='']
                umatan_odds = list(map(lambda x: int(x.replace(',',''))/100 ,umatan_odds_list))
                
                sanrentan_odds_list = sanrentan_row[2].str.split('br').values[0][0:3]
                sanrentan_odds_list = [i for i in sanrentan_odds_list if i!='']
                sanrentan_odds = list(map(lambda x: int(x.replace(',',''))/100 ,sanrentan_odds_list))
        
                umaren_odds = int(umaren_row[2])/100
                sanrenpuku_odds = int(sanrenpuku_row[2])/100
                fukusho_odds_list = fukusho_row[2].str.split('br').values[0][0:3]
                fukusho_odds_list = [i for i in fukusho_odds_list if i!='']
                fukusho_odds = list(map(lambda x: int(x.replace(',',''))/100 , fukusho_odds_list))
                
                wide_odds = list(map(lambda x: int(x.replace(',',''))/100 , wide_row[2].str.split('br').values[0][0:3]))
                
                tmp_list = list(map(lambda x:x.replace(' - ',' '),wide_row[1].str.split('br').values[0][0:3]))
                wide_comb = []
                for tl in tmp_list:
                    pair_list = list(map(lambda x: int(x),tl.split(' ')))
                    wide_comb.append(pair_list)
                    
            # S2
            elif int(umaren_row[1].str.count('br'))==1:
                actual_tmp0 = sanrentan_row[1].str.split('br').values[0][0]
                actual_tmp1 = sanrentan_row[1].str.split('br').values[0][1]
                actual_rank_list0 = list(map(int,actual_tmp0.split('→')))
                actual_rank_list1 = list(map(int,actual_tmp1.split('→')))
                actual_rank_list = [actual_rank_list0,actual_rank_list1]
                
                tansho_odds = int(tansho_row[2])/100
                fukusho_odds_list = fukusho_row[2].str.split('br').values[0][0:3]
                fukusho_odds_list = [i for i in fukusho_odds_list if i!='']
                fukusho_odds = list(map(lambda x: int(x.replace(',',''))/100 , fukusho_odds_list))
                
                umaren_odds_list = umaren_row[2].str.split('br').values[0][0:3]
                umaren_odds_list = [i for i in umaren_odds_list if i!='']
                umaren_odds = list(map(lambda x: int(x.replace(',',''))/100 ,umaren_odds_list))
                
                umatan_odds_list = umatan_row[2].str.split('br').values[0][0:3]
                umatan_odds_list = [i for i in umatan_odds_list if i!='']
                umatan_odds = list(map(lambda x: int(x.replace(',',''))/100 ,umatan_odds_list))
                
                wide_odds = list(map(lambda x: int(x.replace(',',''))/100 , wide_row[2].str.split('br').values[0][0:3]))
                
                tmp_list = list(map(lambda x:x.replace(' - ',' '),wide_row[1].str.split('br').values[0][0:3]))
                wide_comb = []
                for tl in tmp_list:
                    pair_list = list(map(lambda x: int(x),tl.split(' ')))
                    wide_comb.append(pair_list)
                    
                sanrenpuku_odds = int(sanrenpuku_row[2])/100
                sanrentan_odds_list = sanrentan_row[2].str.split('br').values[0][0:3]
                sanrentan_odds_list = [i for i in sanrentan_odds_list if i!='']
                sanrentan_odds = list(map(lambda x: int(x.replace(',',''))/100 ,sanrentan_odds_list))
            
            # S3
            elif int(sanrenpuku_row[1].str.count('br'))==1:
                actual_tmp0 = sanrentan_row[1].str.split('br').values[0][0]
                actual_tmp1 = sanrentan_row[1].str.split('br').values[0][1]
                actual_rank_list0 = list(map(int,actual_tmp0.split('→')))
                actual_rank_list1 = list(map(int,actual_tmp1.split('→')))
                actual_rank_list = [actual_rank_list0,actual_rank_list1]
                
                tansho_odds = int(tansho_row[2])/100
                fukusho_odds_list = fukusho_row[2].str.split('br').values[0][0:4]
                fukusho_odds_list = [i for i in fukusho_odds_list if i!='']
                fukusho_odds = list(map(lambda x: int(x.replace(',',''))/100 , fukusho_odds_list))
                umaren_odds = int(umaren_row[2])/100
                umatan_odds = int(umatan_row[2])/100
                
                wide_odds = list(map(lambda x: int(x.replace(',',''))/100 , wide_row[2].str.split('br').values[0][0:5]))
                tmp_list = list(map(lambda x:x.replace(' - ',' '),wide_row[1].str.split('br').values[0][0:5]))
                wide_comb = []
                for tl in tmp_list:
                    pair_list = list(map(lambda x: int(x),tl.split(' ')))
                    wide_comb.append(pair_list)
                
                sanrenpuku_odds_list = sanrenpuku_row[2].str.split('br').values[0][0:3]
                sanrenpuku_odds_list = [i for i in sanrenpuku_odds_list if i!='']
                sanrenpuku_odds = list(map(lambda x: int(x.replace(',',''))/100 ,sanrenpuku_odds_list))
                
                sanrentan_odds_list = sanrentan_row[2].str.split('br').values[0][0:3]
                sanrentan_odds_list = [i for i in sanrentan_odds_list if i!='']
                sanrentan_odds = list(map(lambda x: int(x.replace(',',''))/100 ,sanrentan_odds_list))
            else:
                actual_rank_list = list(map(int,sanrentan_row[1].str.split('→').values[0]))
                
                tansho_odds = int(tansho_row[2])/100
                umaren_odds = int(umaren_row[2])/100
                sanrenpuku_odds = int(sanrenpuku_row[2])/100
                fukusho_odds_list = fukusho_row[2].str.split('br').values[0][0:3]
                fukusho_odds_list = [i for i in fukusho_odds_list if i!='']
                fukusho_odds = list(map(lambda x: int(x.replace(',',''))/100 , fukusho_odds_list))
                
                wide_odds = list(map(lambda x: int(x.replace(',',''))/100 , wide_row[2].str.split('br').values[0][0:3]))
                
                tmp_list = list(map(lambda x:x.replace(' - ',' '),wide_row[1].str.split('br').values[0][0:3]))
                wide_comb = []
                for tl in tmp_list:
                    pair_list = list(map(lambda x: int(x),tl.split(' ')))
                    wide_comb.append(pair_list)
                
                umatan_odds = int(umatan_row[2])/100
                
                sanrentan_odds = int(sanrentan_row[2])/100
                
        except Exception as e:
            print(e)
            print(race_id)
            return
        
        
        return  pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb,odds_list




In [162]:
sl = Simulater(ll.model)

race_id_list = list(set(ll.x_test.index))
data_c = ll.r.data_c.loc[ll.x_test.index[0]:]

all_results = sl.get_result_df(data_c, return_tables)



In [188]:
length = len(all_results)
tekichu = 0
profit = 0
bet = 100
odds_alpha = 6
race_hit_dist = {'{}'.format(str(i).zfill(2)):0 for i in range(1,13)}


for race_id in all_results.index:
    ar = all_results.loc[race_id]
    pred_list = ar['pred_list']
    actual_list = ar['actual_rank_list']
    tansho_odds = ar['tansho_odds']
    pred_odds = ar['odds_list'][0]
    
    if pred_odds>=odds_alpha:
        profit -= bet
        
    if pred_list[0]==actual_list[0]:
        tekichu+=1
        profit += bet*tansho_odds

print('的中率 {0}'.format(tekichu/length))
print("収益   {0} 円".format(profit))
    

的中率 0.22687224669603523
収益   102230.0 円


# LearnXGB

In [185]:
class LearnLGBM():
    
    
    def __init__(self,peds,results,horse_results):
        self.model = None
        self.model_ft = None
        self.date = '2022/12/31'
        self.pe = None
        self.r = None
        self.horse_results = None
        self.peds = peds
        self.results = results
        self.horse_results = horse_results
        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None
        self.path_ft = '/Users/Owner/Desktop/Horse/horse/peds_ft.txt'

        self.lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 10,
                'learning_rate': 0.02273417953255777,
                'n_estimators': 97,
                'num_leaves': 42,
                'force_col_wise':True
            }



    def learn_model_ft(self,minn=2,maxn=14):
        path_ft = self.path_ft
        model_ft = ft.train_unsupervised(path_ft,dim=62,minn=minn,maxn=maxn)
        self.model_ft = model_ft


    def get_model_ft(self):
        return self.model_ft
    

    def process_pe(self,peds):
        pe = Peds(peds)
        pe.regularize_peds()
        # 血統データ　カテゴリ変数処理
        pe.categorize()
        # pe.vectorize(pe.peds_re,self.model_ft)
        self.pe = pe
        print("pe finish")
        print("pe regularizrd")


    def process_hr(self,results,horse_results):
        r = Results(results)
        r.preprocessing()
        #馬の過去成績データ追加
        hr = HorseResults(horse_results)
        self.hr = hr
        r.merge_horse_results(hr)
        r.merge_peds(self.pe.peds_cat)
        # r.merge_peds(pe.peds_cat)
        #カテゴリ変数の処理
        # pedsは既にカテゴリ化したdataをconcatしているので, ここでカテゴリ化せずとも良い
        r.process_categorical()
        self.r = r


    def process_data(self):
        peds = self.peds.copy()
        results = self.results.copy()
        horse_results = self.horse_results.copy()
        # self.learn_model_ft()
        self.process_pe(peds)
        self.process_hr(results,horse_results)
        
        
    def get_train_data(self,test_size=0.2):
        self.process_data()
        train, test = split_data(self.r.data_c.fillna(0),test_size=test_size,rank_learning=False)
        x_train = train.drop(['rank', 'date','単勝'], axis=1)
        y_train = train['rank']
        x_test = test.drop(['rank', 'date','単勝'], axis=1)
        y_test = test['rank']
        train_query = x_train.groupby(x_train.index).size()
        test_query = x_test.groupby(x_test.index).size()
        train = lgb.Dataset(x_train, y_train, group=train_query)
        test = lgb.Dataset(x_test, y_test, reference=train, group=test_query)
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        return train, test

    
    def get_train_data2(self):
        x_train = self.x_train
        x_test = self.x_test
        y_train = self.y_train
        y_test = self.y_test
        train_query = x_train.groupby(x_train.index).size()
        test_query = x_test.groupby(x_test.index).size()
        train = lgb.Dataset(x_train, y_train, group=train_query)
        test = lgb.Dataset(x_test, y_test, reference=train, group=test_query)
        return train, test


    def learn_lgb(self,lgbm_params=None,test_size=0.2):
        if lgbm_params==None:
            lgbm_params = self.lgbm_params
        
        train, test = self.get_train_data(test_size=test_size)
        lgb_rank = lgb.train(
                lgbm_params,
                train,
                # valid_sets=test,
                num_boost_round=100,
                valid_names=['train'],
                # early_stopping_rounds=20,
            )

        self.model = lgb_rank

# train data を与えて学習させるのが learn_lgb2
    def learn_lgb2(self,train,lgbm_params=None):
        if lgbm_params==None:
            lgbm_params = self.lgbm_params
        
        lgb_rank = lgb.train(
                lgbm_params,
                train,
                # valid_sets=test,
                num_boost_round=100,
                valid_names=['train'],
                # early_stopping_rounds=20,
            )

        self.model = lgb_rank
    

class Predictor(LearnLGBM):


    def __init__(self,peds,results,horse_results,race_id_list):
        super(Predictor, self).__init__(peds,results,horse_results)
        self.race_id_list = race_id_list
        self.nopeds_id_list = []
        

    def process_hr(self,results,horse_results):
        r = Results(results)
        r.preprocessing()
        horse_id_list = self.data['horse_id'].astype(str).unique()
        horse_results_tmp = HorseResults.scrape(horse_id_list)
        new_horse_results = update_data(horse_results,horse_results_tmp)
        self.hores_results = new_horse_results.copy()
        hr = HorseResults(new_horse_results)
        self.hr = hr
        print("hr finish")
        r.merge_horse_results(hr)
        r.merge_peds(self.pe.peds_cat)
        r.process_categorical()  
        self.r = r
        

    def process_data(self):
        race_id_list = self.race_id_list.copy()
        data =  ShutubaTable.scrape(race_id_list, self.date)
        self.data = data
        peds = self.peds.copy()
        results = self.results.copy()
        horse_results = self.horse_results.copy()
        nopeds_id_list = []


        for ind in data['horse_id'].astype(int).unique():
            if ind not in peds.index:
                nopeds_id_list.append(str(ind))

        if len(nopeds_id_list)!=0:
            peds_tmp = Peds.scrape(nopeds_id_list)
            pe_tmp = Peds(peds_tmp)
            pe_tmp.regularize_peds() 
            new_peds = update_data(peds, pe_tmp.peds_re)
        else:
            new_peds = peds.copy()
        
        self.nopeds_id_list = nopeds_id_list
        self.peds = new_peds.copy()
        path_ft =  self.path_ft 
        
        new_peds.to_csv(path_ft,header=False,index=False,sep=',')
        self.learn_model_ft()
        self.process_pe(new_peds)
        self.process_hr(results,horse_results)

    
    def predict(self, race_id):
        data =  ShutubaTable.scrape([str(race_id)], self.date)
        st = ShutubaTable(data)
        st.preprocessing()
        st.merge_horse_results(self.hr)
        st.merge_peds(self.pe.peds_cat)
        st.process_categorical(self.r.le_horse, self.r.le_jockey, self.r.data_pe)
        self.st = st
        sl = Simulater(self.model)
        pred_table = sl.return_pred_table(st.data_c)
        self.sl = sl
        print(pred_table)
            

    def show_results_today(self):
        data =  ShutubaTable.scrape(self.race_id_list, self.date)
        self.data = data.copy()
        st = ShutubaTable(self.data)
        st.preprocessing()
        st.merge_horse_results(self.hr)
        st.merge_peds(self.pe.peds_cat)
        st.process_categorical(self.r.le_horse, self.r.le_jockey, self.r.data_pe)
        sl = RankSimulater(self.model)
        sl.return_table_today(self.race_id_list)
        sl.show_results_today(st ,self.race_id_list)


    

# grid search

In [159]:
train_data = ll.r.data_c
train, valid = split_data(train_data,test_size=0.2,rank_learning=False)
# 0.8 : 0.2

x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_valid = valid.drop(['rank', 'date','単勝'], axis=1)
y_valid = valid['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
valid_query = x_valid.groupby(x_valid.index).size()
valid = lgb.Dataset(x_valid, y_valid, reference=train, group=valid_query)


def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用

    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        'num_iterations': hp.choice('num_iterations',range(50,120)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)


trials = Trials()
optimize(trials)


Training start:                                       
  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76081                    
[LightGBM] [Info] Number of data points in the train set: 96940, number of used features: 172
Training until validation scores don't improve for 20 rounds
  0%|          | 0/25 [00:01<?, ?trial/s, best loss=?]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')



[5]	valid's ndcg@1: 0.527604	valid's ndcg@2: 0.509235	valid's ndcg@3: 0.503558
[10]	valid's ndcg@1: 0.517359	valid's ndcg@2: 0.502096	valid's ndcg@3: 0.497436
[15]	valid's ndcg@1: 0.512237	valid's ndcg@2: 0.501435	valid's ndcg@3: 0.497283
[20]	valid's ndcg@1: 0.507684	valid's ndcg@2: 0.504808	valid's ndcg@3: 0.500056
Early stopping, best iteration is:                    
[4]	valid's ndcg@1: 0.525896	valid's ndcg@2: 0.509207	valid's ndcg@3: 0.508317
Training start:                                                                  
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.053449 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76081                                               
[LightGBM] [Info] Number of data points in the train set: 96940, number of used features: 172
Training until validation scores don't improve for 20 rounds                     
[5]	valid's ndcg@1: 0.479226	valid's ndcg@

# milestone

In [ ]:
'lambdarank_truncation_level':  {'lambdarank_truncation_level': 26, 'learning_rate': 0.04043240395979999, 'num_iterations': 48}46, 'learning_rate': 0.023221403531782224, 'num_iterations': 33}

In [160]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 27,
                'learning_rate': 0.04043240395979999,
                'num_iterations': 98,
                # 'lambdarank_truncation_level': 10,
                # 'learning_rate': 0.02273417953255777,
                # n_estimators は num_iterations と同義　default = 100
                # 'n_estimators': 97,
                # 'num_leaves': 42,
                'force_col_wise':True
            }



In [161]:
import warnings

warnings.filterwarnings("ignore")

ll.learn_lgb2(train,lgbm_params=lgbm_params)

[LightGBM] [Info] Total Bins 76081
[LightGBM] [Info] Number of data points in the train set: 96940, number of used features: 172


# Simulate


In [5]:
# !pwd
save_path = '/Users/Owner/Desktop/Horse/ll_obj.pickle'

def save_pickle(save_path,object_):
    with open(save_path, mode="wb") as f:
        pickle.dump(object_, f)

def load_pickle(save_path):
    with open(save_path, mode="rb") as f:
        object_ = pickle.load(f)
    return object_

# save_pickle(save_path,ll)
ll = load_pickle(save_path)
ll.learn_model_ft()


In [36]:



for odds in [1]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    fukusho_df = sl.get_result_df(ll.r.data_c.loc[ll.x_test.index[0]:],return_tables,kaime='fukusho',odds=odds)
    tansho_df = sl.get_result_df(ll.r.data_c.loc[ll.x_test.index[0]:],return_tables,kaime='tansho',odds=odds)



odds 1


# calc results df

In [11]:
def calc_fukusho_acc(fukusho_df,odds=1.1,bet=100):
    race_num = len(fukusho_df)
    count_in3 = 0
    count_list = [0 for _ in range(3)]

    for i in range(len(fukusho_df)):
        col = fukusho_df.iloc[i]
        actual_rank_list = col['actual rank list']
        pred_list = col['pred list']
        fukusho_list = col['fukusho list']
        is_in = False
        visited = False

        for i,pred in enumerate(pred_list):
            is_in =  pred in fukusho_list
            if is_in and not visited:
                count_in3 += 1
                visited = True

            if pred_list[i] in fukusho_list:
                count_list[i] += 1
                
    # 予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合
    acc_in3 = count_in3/race_num * 100
    acc_list = [i/race_num * 100 for i in count_list]

    print("予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合",acc_in3)
    print("予測した3頭のそれぞれの複勝率",acc_list)
    

def calc_tansho_accuracy(result_df,bet=100):
    race_num = len(result_df)
    atari_num = result_df[result_df['is_atari']==True].sum()['is_atari']
    acc = (atari_num/race_num) * 100
    
    buy_num = result_df[result_df['is_buy']==True].sum()['is_buy']
    buy_df = result_df[result_df['is_buy']]
    atari_buy_num = buy_df[buy_df['is_atari']==True].sum()['is_atari']
    # 条件付き精度 (買ったもとで当たった確率)
    acc_cond = (atari_buy_num/buy_num) * 100
    total_expense = buy_num * bet
    return_price = result_df.sum()['profit']
    # 回収率
    profit_rate = (total_expense + return_price)/total_expense * 100
    atari_dist = {i:0 for i in range(1,13)}
    atari_buy_df = buy_df[buy_df['is_atari']==True]

    for race_id in atari_buy_df.index:
        race_num = int(str(race_id)[-2:])
        atari_dist[race_num] += (1/atari_buy_num) * 100
    
    print('acc ',acc)
    print('acc cond',acc_cond)
    print('profit rate',profit_rate)
    print('atari race distribution',atari_dist)



In [37]:
calc_fukusho_acc(fukusho_df)

予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合 86.20302860347728
予測した3頭のそれぞれの複勝率 [50.1402131239484, 41.61525518788559, 38.75490745933819]


In [39]:

calc_tansho_accuracy(tansho_df)


acc  22.994952327537856
acc cond 22.87323943661972
profit rate 104.82816901408452
atari race distribution {1: 10.344827586206895, 2: 8.866995073891625, 3: 8.374384236453201, 4: 9.60591133004926, 5: 8.620689655172413, 6: 8.12807881773399, 7: 10.591133004926107, 8: 7.881773399014777, 9: 7.635467980295565, 10: 6.403940886699506, 11: 5.1724137931034475, 12: 8.374384236453201}


# race_id_list

In [186]:

race_id_list = ['2022040307{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022100407{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022010207{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list = ['2022040304{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022100404{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022010204{}'.format(str(i).zfill(2)) for i in range(1,13)]



# race_id_list += ['2022050308{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022090301{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090302{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090303{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090304{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022020103{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020104{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020105{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020106{}'.format(str(i).zfill(2)) for i in range(1,13)]



# Predictor

In [187]:
pt = Predictor(peds,results,horse_results,race_id_list)
# ********* test_size = 0  : ずっと0にしてなかった...
pt.path_ft = '/Users/Owner/Desktop/Horse/horse/peds_ft.txt'
pt.learn_lgb(lgbm_params=lgbm_params,test_size=0)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/18721 [00:00<?, ?it/s]

pe finish
pe regularizrd


...

hr finish


  0%|          | 0/277 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

[LightGBM] [Info] Total Bins 80267
[LightGBM] [Info] Number of data points in the train set: 120792, number of used features: 172


# 新潟

In [194]:
base  = '2022040307'
pt.predict(base+'03')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

              馬番    scores
202204030703  12 -0.083040
202204030703   7 -0.133025
202204030703   3 -0.273863
202204030703  15 -0.334509
202204030703  13 -0.390595
202204030703   8 -0.402360
202204030703   1 -0.785690
202204030703   4 -1.245195
202204030703   9 -1.265238
202204030703   6 -1.286396
202204030703   5 -1.316701
202204030703  14 -1.579693
202204030703  11 -2.201587
202204030703  10 -2.358785
202204030703   2 -2.416950


# 小倉

In [198]:
base  = '2022100407'
# for i in ['07','08','09','10','11']:
pt.predict(base+'04')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

              馬番    scores
202210040704   2  0.299285
202210040704   5 -0.372617
202210040704  16 -0.376470
202210040704  11 -0.548466
202210040704   9 -0.880869
202210040704   3 -1.064032
202210040704   1 -1.100521
202210040704  13 -1.206352
202210040704  15 -1.214258
202210040704  10 -1.316961
202210040704   6 -1.662008
202210040704   8 -1.989135
202210040704  14 -2.181520
202210040704  12 -2.206638
202210040704   4 -2.257664
202210040704   7 -2.332593


# 札幌

In [ ]:
base  = '2022010207'
pt.predict(base+'06')

# 当日の収支計算

return_tables　と, return_tables_todayで形式が異なるので, 形式をそろえる

# 当日 sim 手順

In [14]:
# data =  ShutubaTable.scrape(race_id_list, pt.date)
# st = ShutubaTable(data)
# st.preprocessing()
# st.merge_horse_results(pt.hr)
# st.merge_peds(pt.pe.peds_vec)
# st.process_categorical(pt.r.le_horse, pt.r.le_jockey, pt.r.data_pe)


tsl = TodaySimulater(pt.model)
tsl.return_table_today(race_id_list)
all_results = tsl.get_result_df( st.data_c, tsl.return_tables,race_id_list, kaime='all',odds=1)
tansho_results = tsl.get_result_df( st.data_c, tsl.return_tables,race_id_list, kaime='tansho',odds=1)

  0%|          | 0/36 [00:00<?, ?it/s]

# previous params

In [16]:
all_results


,pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb
202204030501,"[15, 8, 14, 12, 2, 7, 9, 10, 6, 4, 11, 3, 1, 5...","[15, 8, 12]",3.0,"[1.2, 1.1, 2.2]",2.8,"[1.6, 4.7, 3.5]",7.3,8.3,38.0,"[[8, 15], [12, 15], [8, 12]]"
202204030502,"[1, 15, 7, 13, 3, 14, 5, 10, 8, 11, 6, 2, 12, ...","[15, 5, 1]",5.8,"[1.8, 6.8, 1.3]",88.7,"[25.0, 3.0, 14.0]",170.0,50.2,435.9,"[[5, 15], [1, 15], [1, 5]]"
202204030503,"[1, 7, 12, 9, 15, 5, 13, 14, 2, 4, 11, 3, 10, 8]","[7, 9, 14]",6.9,"[2.1, 4.4, 3.2]",58.4,"[16.5, 8.9, 25.5]",109.5,164.0,942.7,"[[7, 9], [7, 14], [9, 14]]"
202204030504,"[3, 11, 10, 7, 6, 1, 4, 9, 8, 2, 5, 13, 14, 12]","[10, 13, 9]",5.6,"[2.0, 6.3, 1.8]",82.4,"[28.2, 5.6, 17.8]",124.4,93.0,706.4,"[[10, 13], [9, 10], [9, 13]]"
202204030505,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[10, 2, 7]",7.5,"[1.5, 1.8, 1.1]",31.1,"[6.4, 2.5, 3.0]",58.0,9.3,106.7,"[[2, 10], [7, 10], [2, 7]]"
202204030506,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[2, 6, 11]",6.6,"[1.9, 2.5, 1.4]",42.2,"[12.3, 4.9, 5.5]",69.0,40.0,266.0,"[[2, 6], [2, 11], [6, 11]]"
202204030507,"[18, 14, 15, 12, 16, 5, 13, 11, 10, 4, 3, 7, 2...","[12, 16, 4]",6.5,"[2.0, 1.8, 2.2]",19.9,"[6.8, 7.2, 7.5]",32.6,39.6,247.6,"[[12, 16], [4, 12], [4, 16]]"
202204030508,"[1, 12, 8, 15, 9, 7, 13, 11, 4, 5, 10, 6, 14, ...","[9, 13, 12]",4.6,"[2.0, 5.8, 1.7]",69.7,"[20.4, 3.9, 13.2]",94.9,57.9,473.8,"[[9, 13], [9, 12], [12, 13]]"
202204030509,"[14, 10, 6, 9, 4, 7, 11, 8, 12, 13, 2, 5, 1, 3...","[8, 13, 6]",1.7,"[1.1, 4.6, 1.4]",23.8,"[8.1, 2.1, 18.7]",29.8,32.5,136.6,"[[8, 13], [6, 8], [6, 13]]"
202204030510,"[2, 7, 1, 13, 5, 15, 11, 12, 4, 6, 8, 3, 10, 9...","[13, 11, 5]",3.8,"[1.5, 1.9, 2.2]",11.3,"[4.5, 5.8, 8.3]",19.8,29.9,123.7,"[[11, 13], [5, 13], [5, 11]]"


# 日付に注意

In [47]:
date = '2022/12/31'

# Results scraping

In [48]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

  0%|          | 0/36 [00:00<?, ?it/s]

# Horse_results scraping

In [49]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)

# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

  0%|          | 0/480 [00:00<?, ?it/s]

# Peds scraping

In [50]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

  0%|          | 0/480 [00:00<?, ?it/s]

  0%|          | 0/480 [00:00<?, ?it/s]

# Return scraping

In [51]:
returns_tmp = Return.scrape(race_id_list)


  0%|          | 0/36 [00:00<?, ?it/s]

# update

In [52]:
new_results = update_data(load_csv(path_win+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_win+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_win+'peds.csv'),peds_tmp)
returns = load_csv(path_win+'return.csv')
returns.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
returns_tmp.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
new_return = update_data(returns,returns_tmp)


# save

In [53]:
new_results.to_csv(path_win+'results.csv')
new_horse_results.to_csv(path_win+'horse_results.csv')
new_peds.to_csv(path_win+'peds.csv')
new_return.to_csv(path_win+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価

In [ ]:
lis_ = [1,2,3,4,True,4]
lis2_ = [2,3,4,5,False,5]
dic_ = {
}
dic_['a'] = lis_
dic_['b'] = lis2_

df_ = pd.DataFrame(dic_).T

In [ ]:
df_

In [ ]:
df_.rename(columns={0:'profit',1:'is_atari',2:'is_buy',3:'actual_rank',4:'not_buy_reason',5:'is_error'})